# Entrenamiento de la SRCNN
Presentado por:
- Luis Torres
- Alexis Madrigal
- Alejandro Tevera

Nuestra SRCNN constará de 3 capas convolucionales, donde la primer capa tendrá 128 entradas, es decir, tendremos 128 filtros convolucionales con un kernel de $9\times 9$, en la segunda capa tendremos 64 filtros y un kernel de $5\times 5$ y finalmente en la última capa, únicamente tendra un filtro convolucional y el tamaño de kernel será de $5\times 5$. 
La idea de tener 3 filtros se puede encontrar en https://arxiv.org/abs/1501.00092 , en donde se indica que los métodos tradicionales de super resolución generalmente se basan en 3 pasos, los cuales son:
1. Extracción de parche y representación
2. Mapeo no lineal
3. Reconstrucción

De esta forma, se pueden tener estas 3 etapas en una única red convolucional.

## Importar librerias
Antes de comenzar, importaremos las librerias de las cuales estaremos haciendo uso durante el entrenamiento de la red neuroanl

In [ ]:
import sys
import keras
import cv2
import numpy
import matplotlib
import skimage
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow.experimental.numpy as tnp

print('Python: {}'.format(sys.version))
print('Keras: {}'.format(keras.__version__))
print('OpenCV: {}'.format(cv2.__version__))
print('NumPy: {}'.format(numpy.__version__))
print('Matplotlib: {}'.format(matplotlib.__version__))
print('Scikit-Image: {}'.format(skimage.__version__))
print('TensorFlow: {}'.format(tf.__version__))

# import the necessary packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.optimizers import adam_v2
from skimage.metrics import structural_similarity as ssim
from matplotlib import pyplot as plt
from IPython.display import clear_output
import keras.preprocessing.image_dataset as kds
import cv2
import numpy as np
import math
import os

# python magic function, displays pyplot figures in the notebook
%matplotlib inline

# List of the GPU available of the machine, this is for use the GPU NVIDIA GeForce 940MX to train the model
physical_devices = tf.config.list_physical_devices("GPU")
if(physical_devices[0].device_type != None):
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print(physical_devices[0])

# Indicate the text size of the matplotlib to plot the history results of the model train
fontText = {'family' : 'Times New Roman',
        'weight' : 'normal',
        'size'   : 28}
fontTitle = {'family' : 'Arial',
        'weight' : 'bold',
        'size'   : 36}
matplotlib.rc('font', **fontText)

## 1. Generar imagenes en baja resolución
Para el entrenamiento de nuestra red, a partir de nuestra **dataset** de entrenamiento en alta resolución, generaremos las imagenes en baja resolución, de manera que obtendremos dos conjuntos de imagenes, uno en baja resolución (Obtenido a partir del conjunto original) y el conjunto de entrenamiento original (Alta resolución).

In [ ]:
factor = 4                                                      # Factor de escalamiento para degradado de imagen original
Folder = 'Factor_{}_RGB'.format(factor)                         # Directorio en donde se almacenaran los resultados

if not os.path.exists(Folder):
    os.makedirs(Folder)
if not os.path.exists(Folder + '/Training_Process'):
    os.makedirs(Folder + '/Training_Process')

# Directorios de donde se obtendran el dataset de entrenamiento (Baja resolución) y el dataset "Target" (Alta resolución)
pathHR = 'TrainHR'
pathLR = Folder + '/TrainLR'
paths = os.listdir(pathHR)
if not os.path.exists(pathLR):
    os.makedirs(pathLR)
    
# Generar a partir de las imagenes en baja resolución las imagenes en alta resolución
for file in paths:
    # Abrir el archivo
    img = cv2.imread(pathHR + '/' + file)

    # Determinar las nuevas dimensiones a partir de las dimensiones originales, dependiendo el factor de degradado
    h, w, _ = img.shape
    new_height = int(h / factor)
    new_width = int(w / factor)

    # Reescalar la imagen - Reducir
    img = cv2.resize(img, (new_width , new_height), interpolation = cv2.INTER_LINEAR)

    # Reescalar la imagen - Ampliar
    img = cv2.resize(img, (w, h), interpolation = cv2.INTER_LINEAR)

    # Guardar la imagen
    cv2.imwrite('{}/{}'.format(pathLR,file), img)

print('Todas las imagenes en baja resolución han sido generadas')
pathsLR = os.listdir(pathLR)

## 2. Preparar conjuntos de entrenamiento y prueba
Cuando se realiza entrenamiento a una red neuronal, es necesario que, dentro del conjunto de entrenamiento, contar con un conjunto para pruebas, esto con el fin de "validar" durante el proceso de entrenamiento como de bien se va efectuando el aprendizaje por parte de la red

In [ ]:
# Utilizar el 10% del dataset de entrenamiento como dataset de pruebas
n = len(pathsLR)
trainN = round(n * .9)

# Generar aleatoreidad en el dataset
randomPaths = np.copy(pathsLR)
np.random.shuffle(randomPaths)

trainPaths = randomPaths[:]
testPaths = randomPaths[trainN:n]

print('Dataset : {} images'.format(len(paths)))
print('test : {} images'.format(len(testPaths)))
print('train : {} images'.format(len(trainPaths)))

## 3. Pre-procesamiento (Callback map) para generar el dataset de entrenamiento
Se asume que se reciben imagenes en $RGB$. Para el entrenamiento de nuestra red, lo realizaremos a través de los 3 canales del espacio de color $RGB$. Para ello primero leemos la imagen en $RGB$, despues realizamos una división entre $255$ a esta imagen transformada (Normalizamos la imagen), esto con el fin de reducir la variación de los valores en nuestra imagen y por lo tanto reducir también la variación en los parámetros de nuestra red neuronal. Para reconstruir la imagen debemos multiplicar la imagen de salida por $255$.

$$Entrada=\frac{RGB}{255}$$
$$Salida=(SalidaSRCNN\times 255)$$

In [ ]:
def LoadImage(file):
    Input,Target = tf.py_function(LoadImageNumpy, [file], [np.float32, np.float32])
    return Input,Target

def LoadImageNumpy(filepath):
    if(isinstance(filepath, tf.Tensor)):
        file = np.array(tnp.array(filepath)).astype(str)
    else:
        file = filepath
    Input = cv2.imread('{}/{}'.format(pathLR, file))
    Target = cv2.imread('{}/{}'.format(pathHR, file))
    Input = cv2.cvtColor(Input, cv2.COLOR_BGR2RGB).astype(np.float32)/255
    Target = cv2.cvtColor(Target, cv2.COLOR_BGR2RGB).astype(np.float32)/255
    return Input,Target

imgLR,imgHR = LoadImage(paths[10])

plt.figure(figsize = (20,20))
plt.subplot(1,2,1)
plt.title('Baja resolución')
plt.imshow(imgLR)
plt.axis('off')
plt.subplot(1,2,2)
plt.title('Alta resolución')
plt.imshow(imgHR)
plt.axis('off')
plt.show()

## 4. Generar Dataset
Se generarán las variables **dataset** para el entrenamiento y validación de la red. Este objeto debe devolver la imagen en baja resolución así como su respectivo par en alta resolución.

In [ ]:
# Generar el dataset mediante la libereria tensorflow
ds_train = tf.data.Dataset.from_tensor_slices(trainPaths)
ds_train = ds_train.map(LoadImage, num_parallel_calls = tf.data.experimental.AUTOTUNE)
ds_train = ds_train.batch(1)

ds_test = tf.data.Dataset.from_tensor_slices(testPaths)
ds_test = ds_test.map(LoadImage, num_parallel_calls = tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(1)

for imgLR,imgHR in ds_train.take(1):
    pass

# Verificar que el dataset devuelve la imagen en baja resolución y su respectiva imagen en alta resolución
plt.figure(figsize = (20,20))
plt.subplot(1,2,1)
plt.title('Baja resolución')
plt.imshow(imgLR[0,:,:,:])
plt.axis('off')
plt.subplot(1,2,2)
plt.title('Alta resolución')
plt.imshow(imgHR[0,:,:,:])
plt.axis('off')
plt.show()

## 5. Generar modelo de la red
 Se generará la red neruoanal convolucional, la cual estará constituida por 3 capas convolucionales como se menciono anteriormente. y una vez generada, se dará comienzo al entrenamiento.

In [ ]:
# define model type
def model():
    SRCNN = Sequential()

    # Agregar las capas del modelo, en orden secuencial
    SRCNN.add(Conv2D(filters=128, kernel_size = (9, 9), kernel_initializer='glorot_uniform',
                     activation='relu', padding='same', use_bias=True, strides = 1, input_shape=(None, None, 3)))
    SRCNN.add(Conv2D(filters=64, kernel_size = (5, 5), kernel_initializer='glorot_uniform',
                     activation='relu', padding='same', use_bias=True))
    SRCNN.add(Conv2D(filters=3, kernel_size = (5, 5), kernel_initializer='glorot_uniform',
                     activation='linear', padding='same', use_bias=True))
    
    # Definir optimizador del modelo, para este caso utilizaremos el optimizados Adam, para capas 1 y 2 lr=3e-3
    # y para la capa 3 lr=3e-4
    adam1 = adam_v2.Adam(learning_rate=0.003, beta_1=0.9, beta_2=0.999, epsilon=1e-9)
    adam2 = adam_v2.Adam(learning_rate=0.0003, beta_1=0.9, beta_2=0.999, epsilon=1e-9)
    optimizers_and_layers = [(adam1, SRCNN.layers[:1]), (adam2, SRCNN.layers[2])]
    adam = tfa.optimizers.MultiOptimizer(optimizers_and_layers)

    # Compilar el modelo
    SRCNN.compile(optimizer=adam, loss='mean_squared_error', metrics=['mean_squared_error'])
    
    return SRCNN

SRCNN = model()
print(SRCNN.summary())

## 6. Verificar predicción de la red
Se puede cargar un modelo pre-entrenado desde la carpeta "Models" o bien utilizar la red que esta siendo entrenada en este Notebook

In [ ]:
# Crear directorio de almacenamiento de entrenamiento de la red
if not os.path.exists(Folder + '/models'):
    os.makedirs(Folder + '/models')

#SRCNN.load_weights(Folder + '/models/model_250epochs_3c.h5')                   # Cargar modelo pre-entrenado

# Cargar imagenes de baja resolución y de alta resolución respectivamente, esto con el fin de comparar los resultados
tmpImgHR = cv2.imread(pathHR + '/t60.bmp')
tmpImgHR = cv2.cvtColor(tmpImgHR, cv2.COLOR_BGR2RGB)
tmpImgLR = cv2.imread(pathLR + '/t60.bmp')
tmpImgLR = cv2.cvtColor(tmpImgLR, cv2.COLOR_BGR2RGB)
tmpImg = tmpImgLR.astype(np.float32)/255
dim = tmpImg.shape

# Preparar entrada para predicción por la red
Input_train = np.zeros((1,dim[0],dim[1],dim[2]))
Input_train[0,:,:,:] = tmpImg[:,:,:]

# Comprobar funcionamiento de la red convolucional creada
salida_test = SRCNN.predict(Input_train, batch_size = 1)

# Pos-procesamiento del resultado de salida de la red
tmp = salida_test[0,:,:,:]

for index,x in np.ndenumerate(tmp):
    if tmp[index] < 0:
        tmp[index] = 0 #-tmp[index]
    if tmp[index] > 1:
        tmp[index] = 1 # - tmp[index]

# Dar formato a imagen RGB con valores de [0,255]
tmp = (tmp*255).astype(np.uint8)

plt.figure(figsize = (20,20))
plt.subplot(1,3,1)
plt.title('Alta Resolución')
plt.imshow(tmpImgHR)
plt.axis('off')
plt.subplot(1,3,2)
plt.title('SRCNN')
plt.imshow(tmp)
plt.axis('off')
plt.subplot(1,3,3)
plt.title('Baja Resolución')
plt.imshow(tmpImgLR)
plt.axis('off')
plt.show()

## 7. Pre-procesamiento de la imagen y Callbacks para monitoreo de entrenamiento de la red
- Como se menciono anteriormente, se debe realizar la división entre $255$ para reducir la variación de parámetros.
- Para el entrenamiento de la red, le pasaremos como _Input_ la imagen "normalizada" que acabamos de transformar en el paso anterior, de manera que el entrenamiento de la red será realizado en los 3 canales RGB

In [ ]:
tmpImg = cv2.imread(pathLR + '/t60.bmp')
tmpImg = cv2.cvtColor(tmpImg, cv2.COLOR_BGR2RGB).astype(np.float32)/255
dim = tmpImg.shape
Input_train = np.zeros((1,dim[0],dim[1],dim[2]))
Input_train[0,:,:,:] = tmpImg[:,:,:]

def TrainObservation(SRCNN,epoch,epoch_start,Input,tmpImg,ResPath):
    output = SRCNN.predict(Input, batch_size=1)
    
    tmp = output[0,:,:,:]
    
    # Pre-procesar la imagen (Metodo 2)
    for index,x in np.ndenumerate(tmp):
        if tmp[index] < 0:
            tmp[index] = 0 #-tmp[index]
        if tmp[index] > 1:
            tmp[index] = 1 #2 - tmp[index]
    
    tmp = cv2.cvtColor((tmp*255).astype(np.uint8), cv2.COLOR_RGB2BGR)
    cv2.imwrite('{}/{}/{}.png'.format(Folder, ResPath, epoch_start + epoch),tmp)

class TrainCallbacks(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if (np.mod(epoch,10) == 0 and epoch != 0 ):
            TrainObservation(self.model,epoch,start_epoch,Input_train,tmpImg,'Training_Process')
            self.model.save('{}/models/model_{}epochs_3c.h5'.format(Folder, start_epoch + epoch), include_optimizer = False)
        if(epoch < 150):
            adam1 = adam_v2.Adam(learning_rate=0.003, beta_1=0.9, beta_2=0.999, epsilon=1e-9)
            adam2 = adam_v2.Adam(learning_rate=0.0003, beta_1=0.9, beta_2=0.999, epsilon=1e-9)
            optimizers_and_layers = [(adam1, SRCNN.layers[:1]), (adam2, SRCNN.layers[2])]
            adam = tfa.optimizers.MultiOptimizer(optimizers_and_layers)
        elif(epoch >= 150):
            adam1 = adam_v2.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-9)
            adam2 = adam_v2.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-9)
            optimizers_and_layers = [(adam1, SRCNN.layers[:1]), (adam2, SRCNN.layers[2])]
            adam = tfa.optimizers.MultiOptimizer(optimizers_and_layers)
        clear_output()

## 8. Comenzar entrenamiento de la red
Se dara comienzo al entrenamiento de la red, para esto, se define a partir de cual "epoca" inicial comenzará el entrenamiento (Esto es útil cuando deseamos continuar el entrenamiento a partir de un modelo previamente entrenado), la cantidad de epocas que se entrenará al modelo y la frecuencia con la que se ejecutara el dataset de pruebas (Verificación con una porción del dataset de imagenes el entrenamiento de la red cuando está esta siendo entrenada).

In [ ]:
# Número de epocas que se entrenara la red
start_epoch = 0
train_epoch = 30
val_freq = 5

# Definir optimizador del modelo, para este caso utilizaremos el optimizados Adam, para capas 1 y 2 lr=1e-3
# y para la capa 3 lr=1e-4
adam1 = adam_v2.Adam(learning_rate=0.003, beta_1=0.9, beta_2=0.999, epsilon=1e-9)
adam2 = adam_v2.Adam(learning_rate=0.0003, beta_1=0.9, beta_2=0.999, epsilon=1e-9)
optimizers_and_layers = [(adam1, SRCNN.layers[:1]), (adam2, SRCNN.layers[2])]
adam = tfa.optimizers.MultiOptimizer(optimizers_and_layers)

# Compilar el modelo
SRCNN.compile(optimizer=adam, loss='mean_squared_error', metrics=['mean_squared_error'])

# Cargar un modelo previamente entrenado, a fin de continuar el entrenamiento a partir de este modelo previo
if(os.path.exists('{}/models/model_{}epochs_3c.h5'.format(Folder,start_epoch))):
    SRCNN.load_weights('{}/models/model_{}epochs_3c.h5'.format(Folder,start_epoch))
else:
    print('No existe un modelo correspondiente a la epoca inicial : {}'.format(start_epoch))

if(train_epoch > 0):
    # Comenzar entrenamiento de la red
    history = SRCNN.fit(ds_train, validation_data=ds_test, epochs=train_epoch, validation_freq=val_freq, callbacks=[TrainCallbacks()])

    # Predecir resultado de imagen de entrada y guardar el resultado
    TrainObservation(SRCNN,train_epoch,start_epoch,Input_train,tmpImg,'Training_Process')

    # Guardar el modelo entrenado para su posterior utilización
    SRCNN.save('{}/models/model_{}epochs_3c.h5'.format(Folder, start_epoch + train_epoch), include_optimizer = False)

    # Graficar los resultados del entrenamiento de la red
    plt.figure(figsize=(30,20))
    #plt.plot(history.history['loss'], label='Loss (training data)')
    plt.plot(history.history['mean_squared_error'], label='MSE (training data)')
    #plt.plot(history.history['val_loss'], label='Loss (validation data)')
    plt.title('Training SRCNN results',fontdict=fontTitle)
    plt.xlabel('Epoch',fontdict=fontTitle)
    plt.legend(loc="best")
    plt.savefig('{}/TrainingHistory_{}-{}_epochs.png'.format(Folder, start_epoch,start_epoch + train_epoch))
    plt.show()

## 9. Verificar resultados obtenidos de la red entrenada
Pasaremos de manera iterativa una imagen de prueba, es decir, definiremos cuantas veces la salida de la red convolucional será nuevamente la entrada de está misma. Es decir, inicialmente pasamos la imagen de prueba en baja resolución, despues el resultado obtenido lo pasaremos como entrada a la red, y asi iterativamente hasta obtener un resultado "mejorado" de la salida.

In [ ]:
def LoadTestImg(path):
    # Cargamos la imagen de prueba
    tmpImg = cv2.imread(path)
    tmpImg = cv2.cvtColor(tmpImg, cv2.COLOR_BGR2RGB).astype(np.float32)/255
    dim = tmpImg.shape
    Input_train = np.zeros((1,dim[0],dim[1],dim[2]))
    Input_train[0,:,:,:] = tmpImg[:,:,:]
    return tmpImg,Input_train

# Generamos la red convolucional
SRCNN = model()

# Epoca de entrenamiento de la red con la cual vamos a comprobar el resultado
epoch_model = 350
iterations = 10

# Especificar directorios
testFile = pathLR + '/t60.bmp'
if not(os.path.exists(Folder + '/Test')):
    os.makedirs(Folder + '/Test')

# Cargar el modelo previamente entrenado, a fin de realizar la comprobación de resultados con este modelo entrenado
if(os.path.exists('{}/models/model_{}epochs_3c.h5'.format(Folder,epoch_model))):
    SRCNN.load_weights('{}/models/model_{}epochs_3c.h5'.format(Folder,epoch_model))
    tmpImg = cv2.imread(testFile)
    cv2.imwrite('{}/Test/0.png'.format(Folder),tmpImg)
    tmpImg,Input_train = LoadTestImg(testFile)
    for i in range(iterations):
        TrainObservation(SRCNN, i + 1, 0, Input_train, tmpImg, 'Test')
        print(i + 1)
        tmpImg,Input_train = LoadTestImg('{}/Test/{}.png'.format(Folder,i+1))
    print('Tarea terminada')
else:
    print('No existe un modelo correspondiente a la epoca inicial : {}'.format(epoch_model))